In [ ]:
#imported the necessary libraries. 
from PIL import Image
import os   
import numpy as np
import pandas as pd
import tensorflow as tf   
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical

In [ ]:
#importing the data from kaggle website to google colab and unzip it
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json 
#downloading the dataset
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
!unzip -q /content/gtsrb-german-traffic-sign.zip -d Data

Saving kaggle.json to kaggle.json
 99% 608M/612M [00:11<00:00, 84.8MB/s]
100% 612M/612M [00:11<00:00, 55.5MB/s]


In [ ]:
# Setting the variables (data, labels and classes)
data = []
labels = []
classes = 43

In [ ]:
# Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join('/content/Data/','Train',str(i))
    images = os.listdir(path)

    for a in images:
            image = Image.open(path + '/'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)

In [ ]:
# Converting the lists (data and labels from above) into numpy arrays (features and labels)
features = np.array(data)
labels = np.array(labels)

In [ ]:
# printing the shape of all features and labels
print(features.shape)
print(labels.shape)

(39209, 30, 30, 3)
(39209,)


In [ ]:
# Splitting the dataset as training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels, 
                                                    train_size = 0.8,
                                                    test_size = 0.2,
                                                    random_state=42)

In [ ]:
# printing the shape of all training and testing datasets after the split
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(31367, 30, 30, 3) (31367,)
(7842, 30, 30, 3) (7842,)


In [ ]:
# resizing both X_train and X_test
X_train.resize(31367,2700)
X_test.resize(7842,2700)

In [ ]:
# Normaliazing the data for SVM
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

In [ ]:
# printing the shape of both X_train and X_test
print(X_train.shape)
print(X_test.shape)

(31367, 2700)
(7842, 2700)


In [ ]:
# Creating blank dataframe to store model scores

df_scores = pd.DataFrame(columns = ['Model', 'Score', 'Value'])
df_model = pd.DataFrame(columns = ['Model','Accuracy (%)','Time (mins)'])

In [ ]:
# Defining SVM model
from sklearn import svm
svm_classifier = svm.NuSVC(nu=0.05,
                           kernel='rbf',
                           gamma=0.00001,
                           random_state=100)

In [ ]:
# Fitting SVM
import time
tic = time.perf_counter()

#X_train, X_test, y_train, y_test
svm_classifier.fit(X_train, y_train)

toc = time.perf_counter()
m_svm, s_svm = divmod((toc - tic), 60)
time_svm=float(str(str(int(m_svm))+"."+str(int(m_svm))))

In [ ]:
# Predicting values for test data
y_pred_svm = svm_classifier.predict(X_test)

In [ ]:
from sklearn import metrics
# Calculating recall, precision, f1 score and accuracy of SVM

recall_svm = metrics.recall_score(y_test, y_pred_svm,average='macro')
df_scores.loc[len(df_scores)] = ["SVM","Recall",recall_svm]

precision_svm = metrics.precision_score(y_test, y_pred_svm,average='macro')
df_scores.loc[len(df_scores)] = ["SVM","Precision",precision_svm]

f1_svm = metrics.f1_score(y_test, y_pred_svm,average='macro')
df_scores.loc[len(df_scores)] = ["SVM","F1",f1_svm]

accuracy_svm=metrics.accuracy_score(y_test,y_pred_svm)
df_scores.loc[len(df_scores)] = ["SVM","Accuracy",accuracy_svm]

df_model.loc[len(df_model)] = ["SVM",accuracy_svm*100,time_svm]
accuracy_svm

0.9612343789849528

In [ ]:
# Classification report for SVM

print("Classification report for SVM classifier %s:\n%s\n"
      % (svm_classifier, metrics.classification_report(y_test, y_pred_svm)))

Classification report for SVM classifier NuSVC(gamma=1e-05, nu=0.05, random_state=101):
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        38
           1       0.95      0.96      0.96       496
           2       0.94      0.96      0.95       450
           3       0.89      0.95      0.92       280
           4       0.93      0.98      0.96       418
           5       0.91      0.95      0.93       364
           6       0.94      0.98      0.96        59
           7       0.94      0.94      0.94       278
           8       0.98      0.92      0.95       301
           9       0.98      0.96      0.97       268
          10       0.98      0.98      0.98       370
          11       0.98      0.98      0.98       236
          12       0.99      0.99      0.99       450
          13       0.99      0.98      0.98       452
          14       0.99      0.95      0.97       162
          15       0.96      0.97      0.96    